In [ ]:
import polars as pl
import numpy as np
import networkx as nx
import random
from math import cos, sin, pi ,sqrt
import rtsvg
rt  = rtsvg.RACETrack()
circles     = [(25,25,20),(80,80,30),(20,100,10),(25,60,5),(60,30,5),(48,48,6),(40,90,6)]
svg_hdr     = f'<svg x="0" y="0" width="512" height="512" viewbox="0 0 128 128"><rect x="0" y="0" width="128" height="128" fill="white"/>'
svg_circles = []
for i in range(len(circles)): svg_circles.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" stroke="#000000" fill="none"/>')
svg_ftr     = '</svg>'
rt.tile([svg_hdr + ''.join(svg_circles) + svg_ftr])

In [2]:
#
# For Debugging Purposes
#
def isedgarVoronoi(self, S, Box=None, pad=10, use_circle_radius=False, merge_threshold=0.1):
    # return bisector of points p0 and p1 -- bisector is ((x,y),(u,v)) where u,v is the vector of the bisector
    def bisector(p0, p1):
        x, y     = (p0[0] + p1[0])/2.0, (p0[1] + p1[1])/2.0
        uv       = self.unitVector((p0, p1))
        pdx, pdy = uv[1], -uv[0]
        return ((x,y), (pdx,pdy))
    # For the circle version
    def bisectorForCircles(c0, c1):
        uv              = self.unitVector((c0, c1))
        x0, y0          = c0[0] + uv[0] * c0[2], c0[1] + uv[1] * c0[2]
        x1, y1          = c1[0] - uv[0] * c1[2], c1[1] - uv[1] * c1[2]
        x,  y           = (x0+x1)/2.0, (y0+y1)/2.0
        pdx, pdy        = uv[1], -uv[0]
        return ((x,y), (pdx,pdy))
    # returns vertices that intersect the polygon
    def intersects(bisects, poly):
        already_added = set()
        inters        = []
        for i in range(0, len(poly)):
            p0, p1 = poly[i], poly[(i+1)%len(poly)]
            xy = self.lineSegmentIntersectionPoint((bisects[0], (bisects[0][0] + bisects[1][0], bisects[0][1] + bisects[1][1])),
                                                    (p0, p1))
            if xy is not None:
                too_close = False
                for pt in already_added:
                    if self.segmentLength((pt, xy)) < merge_threshold:
                        too_close = True
                        print('too close')
                        break
                if too_close == False:
                    inters.append((xy, i, (i+1)%len(poly)))
                    already_added.add(xy)
        return inters
    # conctenate a point, a list, and a point into a new list
    def createCell(my_cell, p0, p1, i0, i1, sign):
        l = [p0]
        i = i0
        while i != i1:
            l.append(my_cell[i])
            i += sign
            if i < 0:             i += len(my_cell)
            if i >= len(my_cell): i -= len(my_cell)
        l.append(my_cell[i1])
        l.append(p1)
        return l
    # contain true if pt is in poly
    def contains(poly, pt):
        inter_count = 0
        for i in range(len(poly)):
            p0, p1 = poly[i], poly[(i+1)%len(poly)]
            _tuple_ = self.segmentsIntersect((pt,(pt[0]+1e9,pt[1])),(p0,p1)) # use a ray from the pt to test
            if _tuple_[0] and (_tuple_[1], _tuple_[2]) != p1: inter_count += 1
        if inter_count%2 == 1: return True
        return False
    # actual algorithm
    def makePicture(_i_, _p_, _q_, _B_, _B_intersects_, _cell_, _newCell_=None, _otherNewCell_=None):
        def polyPath(_poly_):
            _d_ = f'M {_poly_[0][0]} {_poly_[0][1]} '
            for i in range(1,len(_poly_)): _d_ += f'L {_poly_[i][0]} {_poly_[i][1]} '
            return _d_ + 'Z'
        _svg_ = ['<svg x="0" y="0" width="128" height="128" viewbox="0 0 128 128">']
        _svg_.append(f'<rect x="0" y="0" width="128" height="128" fill="white"/>')
        _svg_.append(rt.svgText(str(_i_), 2, 126, 8))
        for _xy_ in S: _svg_.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="2" fill="#a0a0a0"/>')
        _svg_.append(f'<circle cx="{_p_[0]}" cy="{_p_[1]}" r="2" fill="red"/>')
        if _q_ is not None: _svg_.append(f'<circle cx="{_q_[0]}" cy="{_q_[1]}" r="2" fill="black"/>')
        _svg_.append(f'<path d="{polyPath(_cell_)}" stroke="#0000ff" stroke-width="0.2" fill="none"/>')
        _color_ = '#ff0000' if _newCell_ is None else '#000000'
        if _B_ is not None: _svg_.append(f'<line x1="{_B_[0][0]-_B_[1][0]*100}" y1="{_B_[0][1]-_B_[1][1]*100}" x2="{_B_[0][0]+_B_[1][0]*100}" y2="{_B_[0][1]+_B_[1][1]*100}" stroke="{_color_}" stroke-width="0.2"/>')
        _svg_.append('</svg>')
        return ''.join(_svg_)
    def removeDuplicatePoints(_poly_):
        _new_ = []
        _new_.append(_poly_[0])
        for i in range(1,len(_poly_)):
            if rt.segmentLength((_poly_[i],_new_[-1])) > merge_threshold: _new_.append(_poly_[i])
        return _new_
    if Box is None:
        x_l, x_r, y_t, y_b = S[0][0], S[0][0], S[0][1], S[0][1]
        for pt in S: x_l, x_r, y_t, y_b = min(x_l, pt[0]), max(x_r, pt[0]), max(y_t, pt[1]), min(y_b, pt[1])
        Box = [(x_l-pad, y_t+pad), (x_r+pad, y_t+pad), (x_r+pad, y_b-pad), (x_l-pad, y_b-pad)]
    cells, pics_or_it_didnt_happen = [], []
    for i in range(len(S)):
        p       = S[i]
        cell    = Box
        process = []
        for j in range(len(S)):
            q = S[j]
            if p == q: continue
            B            = bisector(p,q)
            B_intersects = intersects(B, cell)
            if i == 8 and j == 9:
                print(B)
                print(cell)
                print(len(B_intersects), B_intersects)
            if len(B_intersects) == 2:
                t1, t2 = B_intersects[0][0], B_intersects[1][0]
                xi, xj = B_intersects[0][2], B_intersects[1][1]
                newCell      = createCell(cell, t1, t2, xi, xj, 1)
                xi, xj = B_intersects[0][1], B_intersects[1][2]
                otherNewCell = createCell(cell, t1, t2, xi, xj, -1)
                if contains(newCell, p) == False: newCell,otherNewCell = otherNewCell,newCell
                process.append(makePicture(i, p, q, B, B_intersects, cell, newCell, otherNewCell))
                cell = removeDuplicatePoints(newCell)
            else:
                process.append(makePicture(i, p, q, B, B_intersects, cell))
        process.append(makePicture(i, p, None, None, None, cell))
        cells.append(cell), pics_or_it_didnt_happen.append(rt.table(process, per_row=8, spacer=10))

    return cells, pics_or_it_didnt_happen


In [ ]:
min_num_cpoints, max_num_cpoints = 8, 72
voronoi_points = []
for i in range(len(circles)):
    x,y,r = circles[i]
    num_cpoints = int(min_num_cpoints + (max_num_cpoints - min_num_cpoints) * r/50.0)
    if num_cpoints > max_num_cpoints: num_cpoints = max_num_cpoints
    for j in range(num_cpoints):
        a = 2*pi*j/num_cpoints
        voronoi_points.append((x+r*cos(a),y+r*sin(a)))
polys         = rt.isedgarVoronoi(    voronoi_points, [(0,0),(0,128),(128,128),(128,0)])
#polys, poidh = isedgarVoronoi(rt, voronoi_points, [(0,0),(0,128),(128,128),(128,0)])
svg_polys    = []
for i in range(len(polys)):
    d = f'M {polys[i][0][0]} {polys[i][0][1]} '
    for j in range(1,len(polys[i])): d += f'L {polys[i][j][0]} {polys[i][j][1]} '
    d += f'Z'
    svg_polys.append(f'<path d="{d}" stroke="#000000" stroke-width="0.1" fill="none"/>')
    svg_polys.append(f'<circle cx="{voronoi_points[i][0]}" cy="{voronoi_points[i][1]}" r="1" fill="red"/>')
    svg_polys.append(rt.svgText(str(i),voronoi_points[i][0],voronoi_points[i][1]+3.5,3,anchor='middle'))
svg_minus_circle_overlaps = []
for i in range(len(polys)):
    _poly_ = polys[i]
    for j in range(len(_poly_)):
        p0, p1 = _poly_[j], _poly_[(j+1)%len(_poly_)]
        within_circle = False
        for k in range(len(circles)):
            x,y,r = circles[k]
            _closest_d_, _closest_xy_ = rt.closestPointOnSegment((p0,p1),(x,y))
            if _closest_d_ < r:
                within_circle = True
                break
        if within_circle == False: svg_minus_circle_overlaps.append(f'<line x1="{p0[0]}" y1="{p0[1]}" x2="{p1[0]}" y2="{p1[1]}" stroke="red" stroke-width="0.1"/>')
    svg_minus_circle_overlaps.append(f'<circle cx="{voronoi_points[i][0]}" cy="{voronoi_points[i][1]}" r="0.2" fill="red"/>')
    
rt.tile([svg_hdr + ''.join(svg_circles) + ''.join(svg_polys)                 + svg_ftr, 
         svg_hdr + ''.join(svg_polys)                                        + svg_ftr,
         svg_hdr + ''.join(svg_circles) + ''.join(svg_minus_circle_overlaps) + svg_ftr], spacer=5)

In [4]:
#rt.tile([poidh[8]], horz=False, spacer=10)